<a href="https://colab.research.google.com/github/2005latha/2005latha/blob/main/livertumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d greedyfornothing/multiclass-semantic-ct-scan-liver-tumor

# 5. Unzip dataset
!unzip multiclass-semantic-ct-scan-liver-tumor.zip -d liver_tumor_data

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Define your dataset path
data_dir = 'liver_tumor_data'
tumor_dir = 'liver_tumor_data/Train_Ct-Scan'     # replace with your actual folder structure
no_tumor_dir =  'liver_tumor_data/Train_Ct-Scan_masks' # replace accordingly

# Load images and labels
images = []
labels = []
IMG_SIZE = 128

for label, folder in enumerate([no_tumor_dir, tumor_dir]):
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)

images = np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
import os

for root, dirs, files in os.walk('liver_tumor_data'):
    print('ROOT:', root)
    print('DIRS:', dirs)
    print('FILES:', files)
    print('-'*20)

In [ ]:
def show_examples(X, y, label_map={0: 'No Tumor', 1: 'Tumor'}):
    plt.figure(figsize=(10, 4))
    for i in range(5):
        idx = np.where(y==i%2)[0][0]
        plt.subplot(1, 5, i+1)
        plt.imshow(X[idx].squeeze(), cmap='gray')
        plt.title(label_map[y[idx]])
        plt.axis('off')
    plt.show()

show_examples(X_train, y_train)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 128  # Set your desired image size

# X_train, y_train = ... # Load your data here

def build_cnn():
    model = models.Sequential([
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),  # Use Input layer
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_ crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn()
history = cnn_model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=32)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = cnn_model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred, target_names=['No Tumor', 'Tumor']))

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_flat, y_train)
y_lr_pred = lr.predict(X_test_flat)

print("Logistic Regression Results:")
print(classification_report(y_test, y_lr_pred, target_names=['No Tumor', 'Tumor']))

In [ ]:
def plot_class_images(X, y, class_label, title):
    idxs = np.where(y==class_label)[0][:5]
    plt.figure(figsize=(10,2))
    for i, idx in enumerate(idxs):
        plt.subplot(1, 5, i+1)
        plt.imshow(X[idx].squeeze(), cmap='gray')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

plot_class_images(X_test, y_test, 1, "Tumor X-rays/Scans")
plot_class_images(X_test, y_test, 0, "No Tumor X-rays/Scans")